In [ ]:
from textblob import TextBlob
import pandas as pd
import numpy as np
import httpcore,os,json,time

In [ ]:
class TraducaoMqV():
    def __init__(self,df):
        self.df = df
        pergunta_um = []
        pergunta_dois = []
    
    def fun_translater(pergunta):         
        loop = True
        while(loop):
            try:
                translator = TextBlob(pergunta)
                pergunta_br = translator.translate(to = 'pt')
                time.sleep(5)
                loop = False # Stop the loop
                return pergunta_br

            except Exception:
                time.sleep(10)
            
    def init_traducao(self):
        for i in range(463,self.df.shape[0]):
            try:
                pergunta_um_traduzida = self.fun_translater(self.df.loc[i][1])
                pergunta_dois_traduzida = self.fun_translater(self.df.loc[i][2])
                self.pergunta_um.append(pergunta_um_traduzida.string)
                self.pergunta_dois.append(pergunta_dois_traduzida.string)
                print("debug 1")

                with open('./backup/pergunta_um.json','w+') as outfile_um:
                    json.dump(self.pergunta_um,outfile_um,ensure_ascii=False)
                    outfile.close()

                with open('./backup/pergunta_dois.json','w+') as outfile_dois:
                    json.dump(self.pergunta_dois,outfile_dois,ensure_ascii=False)
                    outfile.close()

                f = open('output.txt','a')
                f.write("%d de %d\n"%(i,self.df.shape[0]))
                f.close()

                df_ = pd.DataFrame({'pergunta_um' : self.pergunta_um, 'pergunta_dois' : self.pergunta_dois})
                df_.to_csv('./backup/backupMQP.csv',index=False)
                
                print("%d de %d"%(i,self.df.shape[0]))

            except (KeyboardInterrupt or httpcore._exceptions.ConnectError) as x:

                f = open('output.txt','a')
                f.write("Traducao parada na posicao %d\n"%(i))
                f.write(str(x))
                f.close()

        df_save = pd.DataFrame({'pergunta_um' : self.pergunta_um, 'pergunta_dois' : self.pergunta_dois, 'label' : self.df['label']})
        df_save.to_csv('./mqp.csv',index=False)
        f = open('output.txt','a')
        f.write("Traducao concluida\n")
        f.close()
        
if('__main__' == __name__):

    df = pd.read_csv('./mqp.csv',names = ['dr_id', 'question_1', 'question_2', 'label'])

    path = './backup'
    if(not os.path.exists(path)):
        os.mkdir(path)
    
    print('oido')
    traducao_mqv = TraducaoMqV(df)
    traducao_mqv.init_traducao()

In [8]:
from textblob import TextBlob
from pandas import*
import os,json,time

In [ ]:
def fun_translater(texto):
    pergunta_br = []
    for pergunta in texto:
        loop = True
        while(loop):
            try:
                blob = TextBlob(pergunta)
                pergunta_br.append(blob.translate(from_lang='en',to='pt').string)
                time.sleep(5)
                loop = False # Stop the loop

            except:
                time.sleep(10)
    return pergunta_br

if('__main__' == __name__):

    df = read_csv('./mqp.csv',names = ['dr_id', 'question_1', 'question_2', 'label'])

    path = './backup'
    if(not os.path.exists(path)):
        os.mkdir(path)


    pergunta_um = []
    pergunta_dois = []

    for i in range(463,df.shape[0]):
        texto = [df['question_1'].values[i],df['question_2'].values[i]]
        try:
            traduzido = fun_translater(texto)
            pergunta_um.append(traduzido[0])
            pergunta_dois.append(traduzido[1])
            
        except:
            f = open('output.txt','a')
            f.write("Traducao parada na posicao %d\n"%(i))
            f.close()   
        
        finally:
            
            with open('./backup/pergunta_um.json','w') as outfile:
                json.dump(pergunta_um,outfile,ensure_ascii=False)
                outfile.close()

            with open('./backup/pergunta_dois.json','w') as outfile:
                json.dump(pergunta_dois,outfile,ensure_ascii=False)
                outfile.close()

            f = open('output.txt','a')
            f.write("%d de %d\n"%(i + 1,df.shape[0]))
            f.close()

            DataFrame({'pergunta_um' : pergunta_um, 'pergunta_dois' : pergunta_dois}).to_csv('./backup/backupMQP.csv',index=False)
            

        

    DataFrame({'pergunta_um' : pergunta_um, 'pergunta_dois' : pergunta_dois, 'label' : df['label']}).to_csv('./mqp.csv',index=False)
    f = open('output.txt','a')
    f.write("Traducao concluida\n")
    f.close()
